In [19]:
from mindformers.core.loss import CrossEntropyLoss

In [20]:
loss1 = CrossEntropyLoss()

In [21]:
import mindspore.numpy as msnp

In [22]:
x = msnp.randn(992, 30523)

In [23]:
y = msnp.randn(992).astype(ms.int32)

In [1]:
import mindspore as ms
ms.set_context(device_id=7)

In [2]:
x = ms.load_checkpoint("/home/yj/QFormer/shifted_prediction_scores.ckpt")['shifted_prediction_scores'].value()

In [3]:
x.shape

(992, 30523)

In [4]:
y = ms.load_checkpoint("/home/yj/QFormer/labels.ckpt")['labels'].value()

In [5]:
y.shape

(992,)

In [6]:
import mindspore.ops as ops

In [7]:
import mindspore.ops.operations as P

In [15]:
_gather_d = P.GatherD()

In [11]:
target_dim = -1

In [12]:
y1 = y.expand_dims(target_dim)

In [13]:
loss = _gather_d(x, target_dim, y1)

In [14]:
loss

Tensor(shape=[992, 1], dtype=Float32, value=
[[ 9.77265418e-01],
 [ 6.47866130e-01],
 [ 7.58817554e-01],
 ...
 [ 1.80141628e-02],
 [-6.06357396e-01],
 [ 5.73751628e-02]])

In [30]:
y_x = ops.arange(y.shape[0]) * x.shape[-1]

In [27]:
y_mod = ops.floor_mod(y, 30523)

In [32]:
y_mod = y_x + y_mod

In [35]:
loss2 = x.flatten()[y_mod].expand_dims(target_dim)

In [38]:
loss2

Tensor(shape=[992, 1], dtype=Float32, value=
[[ 9.77265418e-01],
 [ 6.47866130e-01],
 [ 7.58817554e-01],
 ...
 [ 1.80141628e-02],
 [-6.06357396e-01],
 [ 5.73751628e-02]])

In [8]:
import mindspore.nn as nn
import mindspore.numpy as msnp
class CrossEntropyLossV2(nn.Cell):
    """
    Calculate the cross entropy loss.
    """
    def __init__(self, target_dim=-1, weight=None, ignore_index=-100, reduction='mean', label_smoothing=0.0):
        super(CrossEntropyLossV2, self).__init__()   
        self.target_dim = target_dim
        self.weight = weight
        self.ignore_index = ignore_index
        self.reduction = reduction
        self.label_smoothing = label_smoothing
        
        self.exp = P.Exp()
        self.log = P.Log()
        self.neg = P.Neg()
        self.gather = P.Gather()
        self.ones_like = P.OnesLike()
        self.equal = P.Equal()


    def logsumexp(self, x, axis, keep_dims=False):
        """
        Reduces a dimension of a tensor by calculating exponential for all elements in the dimension,
        then calculate logarithm of the sum.
        """
        _reduce_sum = P.ReduceSum(keep_dims)

        x_max = x.max(axis=axis, keepdims=True)
        x_exp = self.exp(x - x_max)
        x_sumexp = _reduce_sum(x_exp, axis)
        x_logsumexp = self.log(x_sumexp)
        if not keep_dims:
            x_max = x_max.squeeze(axis=axis)
        return x_logsumexp + x_max

    def log_softmax(self, inputs, axis):
        """inner implementation of log_softmax, since the LogSoftmaxGrad op do not support inputs > 2d"""
        return inputs - self.logsumexp(inputs, axis, True)

# def nll_loss(inputs, target, weight=None, ignore_index=-100, reduction='mean', label_smoothing=0.0):
#     """
#     Gets the negative log likelihood loss between inputs and target.
#     """
#     ndim = inputs.ndim
#     if ndim == 2:
#         ret = _nll_loss(inputs, target, -1, weight, ignore_index, reduction, label_smoothing)
#     elif ndim == 4:
#         ret = _nll_loss(inputs, target, 1, weight, ignore_index, reduction, label_smoothing)
#     elif ndim == 1:
#         ret = _nll_loss(inputs, target, 0, weight, ignore_index, reduction, label_smoothing)
#     else:
#         n = inputs.shape[0]
#         c = inputs.shape[1]
#         out_size = (n,) + inputs.shape[2:]
#         inputs = inputs.view(n, c, 1, -1)
#         target = target.view(n, 1, -1)
#         if reduction != 'none':
#             ret = _nll_loss(inputs, target, 1, weight, ignore_index, reduction, label_smoothing)
#         else:
#             ret = _nll_loss(inputs, target, 1, weight, ignore_index, label_smoothing=label_smoothing)
#             ret = ret.view(out_size)
#     return ret

    def Gather_D(self, inputs, target_dim, target):
        """
        Rewrite P.GatherD(), align with it.
        """
        pred_x = msnp.arange(target.shape[0]) * inputs.shape[-1]
        pred_mod = ops.floor_mod(target, inputs.shape[-1])
        pred_idx = pred_x + pred_mod
        return (inputs.flatten())[pred_idx].expand_dims(target_dim)

    def nll_loss(self, inputs, target, target_dim=-1, weight=None, ignore_index=None, reduction='none', label_smoothing=0.0):
        """nll loss inner function"""
        if target.ndim == inputs.ndim - 1:
            target = target.expand_dims(target_dim)
        if ignore_index is not None:
            non_pad_mask = self.equal(target, ignore_index)
            target = target.masked_fill(non_pad_mask, 0)
        else:
            non_pad_mask = target
        target = target.squeeze(target_dim)
        loss = self.neg(self.Gather_D(inputs, target_dim, target))
        smooth_loss = self.neg(inputs.sum(axis=target_dim, keepdims=False))

        if weight is not None:
            loss_weights = self.gather(weight, target, 0)
            loss = loss * loss_weights
        else:
            loss_weights = self.ones_like(loss)
        if ignore_index is not None:
            loss = loss.masked_fill(non_pad_mask, 0.)
            loss_weights = loss_weights.masked_fill(non_pad_mask, 0.)
    #         smooth_loss = smooth_loss.masked_fill(non_pad_mask, 0.)

        loss = loss.squeeze(target_dim)
    #     smooth_loss = smooth_loss.squeeze(target_dim)

        if reduction == 'sum':
            loss = loss.sum()
            smooth_loss = smooth_loss.sum()
        if reduction == 'mean':
            loss = loss.sum() / loss_weights.sum()
    #         loss = loss.mean()
            smooth_loss = smooth_loss.mean()

        loss = (1. - label_smoothing) * loss + label_smoothing * smooth_loss / inputs.shape[target_dim]
        return loss
    
    
    def construct(self, inputs, target):
        r"""
        The cross entropy loss between input and target.
        """
        class_dim = 0 if inputs.ndim == 1 else 1
        log_softmax_result = self.log_softmax(inputs, class_dim)
    #     print("log_softmax_result: ", log_softmax_result)
        return self.nll_loss(log_softmax_result, target, self.target_dim, self.weight, self.ignore_index, self.reduction, self.label_smoothing)

In [9]:
cross_entropy_v2 = CrossEntropyLossV2(label_smoothing=0.1)

In [10]:
cross_entropy_v2(x, y)

Tensor(shape=[], dtype=Float32, value= 10.2473)

In [87]:
import torch
from torch.nn import CrossEntropyLoss as torch_CrossEntropyLoss
default_torch_crossentropy = torch_CrossEntropyLoss(label_smoothing=0.1)
def compute_torch_CrossEntropy(x, y, torch_crossentropy=default_torch_crossentropy):
    x_torch = torch.from_numpy(x.asnumpy())
    y_torch = torch.from_numpy(y.asnumpy()).to(torch.long)
    return torch_crossentropy(x_torch, y_torch).detach().numpy()

In [92]:
import numpy as np
for i in range(round_n):
    x_rand = msnp.randn(x.shape)
    y_rand = msnp.randn(y.shape).astype(ms.int32)
    ms_loss = cross_entropy_v2(x + x_rand, y).asnumpy()
    torch_loss = compute_torch_CrossEntropy(x + x_rand, y)
    print(f"ms_loss: {ms_loss:.5f}\b torch_loss: {torch_loss:.5f}\b l1_error: {np.abs(ms_loss - torch_loss):.6f}")

ms_loss: 10.7286 torch_loss: 10.7286 l1_error: 0.000004
ms_loss: 10.6643 torch_loss: 10.6643 l1_error: 0.000058
ms_loss: 10.7569 torch_loss: 10.7570 l1_error: 0.000021
ms_loss: 10.7431 torch_loss: 10.7431 l1_error: 0.000030
ms_loss: 10.7696 torch_loss: 10.7697 l1_error: 0.000065
ms_loss: 10.6608 torch_loss: 10.6608 l1_error: 0.000011
ms_loss: 10.7278 torch_loss: 10.7277 l1_error: 0.000037
ms_loss: 10.7249 torch_loss: 10.7250 l1_error: 0.000076
ms_loss: 10.7421 torch_loss: 10.7421 l1_error: 0.000014
ms_loss: 10.7658 torch_loss: 10.7658 l1_error: 0.000007
ms_loss: 10.7310 torch_loss: 10.7311 l1_error: 0.000044
ms_loss: 10.7179 torch_loss: 10.7179 l1_error: 0.000015
ms_loss: 10.7519 torch_loss: 10.7520 l1_error: 0.000045
ms_loss: 10.8749 torch_loss: 10.8749 l1_error: 0.000014
ms_loss: 10.6635 torch_loss: 10.6636 l1_error: 0.000085
ms_loss: 10.6871 torch_loss: 10.6870 l1_error: 0.000029
ms_loss: 10.7616 torch_loss: 10.7617 l1_error: 0.000101
ms_loss: 10.7621 torch_loss: 10.7621 l1_error: 0

KeyboardInterrupt: 

In [46]:
cross_entropy(x, y, label_smoothing=0.1)

Tensor(shape=[], dtype=Float32, value= 10.2473)

In [41]:
import mindspore.ops.functional as F

In [42]:
F.cross_entropy(x, y, label_smoothing=0)

log_softmax_result:  [[-10.444187 -10.315205 -10.759834 ... -10.003607 -10.650483 -10.019769]
 [-10.113524 -11.206087 -10.332144 ... -10.155675 -10.70844  -10.906179]
 [-11.029074 -10.338369 -10.502667 ... -10.243991 -10.717099 -10.276416]
 ...
 [ -9.84885  -10.186969 -10.872633 ... -10.526638 -10.044142 -11.196835]
 [ -9.912016 -10.702992 -11.336755 ... -10.9684   -10.914676 -11.530894]
 [ -9.825527 -10.772986 -11.110288 ... -10.19183  -10.542184 -10.758883]]


Tensor(shape=[], dtype=Float32, value= 10.2214)

In [11]:
from mindformers.models.blip2.qformer import CrossEntropyLoss as CrossEntropyLoss_ori

In [12]:
cross_entropy_ori = CrossEntropyLoss_ori(label_smoothing=0.1)

In [ ]:
import numpy as np
round_n = 1000
for i in range(round_n):
    x_rand = msnp.randn(x.shape)
    y_rand = msnp.randn(y.shape).astype(ms.int32)
    ms_loss = cross_entropy_v2(x + x_rand, y).asnumpy()
    ori_loss = cross_entropy_ori(x + x_rand, y).asnumpy()
    print(f"ms_loss: {ms_loss:.5f}\b ori_loss: {ori_loss:.5f}\b l1_error: {np.abs(ms_loss - ori_loss):.6f}")

ms_loss: 10.7461 ori_loss: 10.7461 l1_error: 0.000000
ms_loss: 10.7775 ori_loss: 10.7775 l1_error: 0.000000
ms_loss: 10.7705 ori_loss: 10.7705 l1_error: 0.000001
ms_loss: 10.6944 ori_loss: 10.6944 l1_error: 0.000000
ms_loss: 10.7715 ori_loss: 10.7715 l1_error: 0.000000
ms_loss: 10.7896 ori_loss: 10.7896 l1_error: 0.000000
ms_loss: 10.7053 ori_loss: 10.7053 l1_error: 0.000000
ms_loss: 10.7394 ori_loss: 10.7394 l1_error: 0.000001
ms_loss: 10.6897 ori_loss: 10.6897 l1_error: 0.000000
ms_loss: 10.7667 ori_loss: 10.7667 l1_error: 0.000000
ms_loss: 10.7316 ori_loss: 10.7316 l1_error: 0.000001
ms_loss: 10.8103 ori_loss: 10.8103 l1_error: 0.000000
ms_loss: 10.7917 ori_loss: 10.7917 l1_error: 0.000000
ms_loss: 10.7400 ori_loss: 10.7400 l1_error: 0.000000
ms_loss: 10.8030 ori_loss: 10.8030 l1_error: 0.000001
ms_loss: 10.7240 ori_loss: 10.7240 l1_error: 0.000000
ms_loss: 10.7224 ori_loss: 10.7224 l1_error: 0.000000
ms_loss: 10.8546 ori_loss: 10.8546 l1_error: 0.000001
ms_loss: 10.7472 ori_loss: 1

ms_loss: 10.7920 ori_loss: 10.7920 l1_error: 0.000000
ms_loss: 10.7951 ori_loss: 10.7951 l1_error: 0.000000
ms_loss: 10.8571 ori_loss: 10.8571 l1_error: 0.000000
ms_loss: 10.7370 ori_loss: 10.7370 l1_error: 0.000000
ms_loss: 10.8114 ori_loss: 10.8114 l1_error: 0.000000
ms_loss: 10.7832 ori_loss: 10.7832 l1_error: 0.000000
ms_loss: 10.7723 ori_loss: 10.7723 l1_error: 0.000001
ms_loss: 10.7148 ori_loss: 10.7148 l1_error: 0.000000
ms_loss: 10.7187 ori_loss: 10.7187 l1_error: 0.000000
ms_loss: 10.7376 ori_loss: 10.7376 l1_error: 0.000002
ms_loss: 10.7666 ori_loss: 10.7666 l1_error: 0.000000
ms_loss: 10.7400 ori_loss: 10.7400 l1_error: 0.000000
ms_loss: 10.7042 ori_loss: 10.7042 l1_error: 0.000000
ms_loss: 10.7558 ori_loss: 10.7558 l1_error: 0.000000
ms_loss: 10.7581 ori_loss: 10.7581 l1_error: 0.000000
ms_loss: 10.7162 ori_loss: 10.7162 l1_error: 0.000000
ms_loss: 10.6694 ori_loss: 10.6694 l1_error: 0.000000
ms_loss: 10.8175 ori_loss: 10.8175 l1_error: 0.000000
ms_loss: 10.7583 ori_loss: 1

ms_loss: 10.7699 ori_loss: 10.7699 l1_error: 0.000000
ms_loss: 10.7252 ori_loss: 10.7252 l1_error: 0.000000
ms_loss: 10.6117 ori_loss: 10.6117 l1_error: 0.000000
ms_loss: 10.8004 ori_loss: 10.8004 l1_error: 0.000000
ms_loss: 10.7266 ori_loss: 10.7266 l1_error: 0.000000
ms_loss: 10.7641 ori_loss: 10.7641 l1_error: 0.000001
ms_loss: 10.6789 ori_loss: 10.6789 l1_error: 0.000000
ms_loss: 10.6931 ori_loss: 10.6931 l1_error: 0.000000
ms_loss: 10.7343 ori_loss: 10.7343 l1_error: 0.000000
ms_loss: 10.7539 ori_loss: 10.7539 l1_error: 0.000000
ms_loss: 10.7838 ori_loss: 10.7838 l1_error: 0.000000
ms_loss: 10.7284 ori_loss: 10.7284 l1_error: 0.000000
ms_loss: 10.7321 ori_loss: 10.7321 l1_error: 0.000000
ms_loss: 10.6576 ori_loss: 10.6576 l1_error: 0.000000
ms_loss: 10.7966 ori_loss: 10.7966 l1_error: 0.000000
ms_loss: 10.7884 ori_loss: 10.7884 l1_error: 0.000000
ms_loss: 10.7616 ori_loss: 10.7616 l1_error: 0.000000
ms_loss: 10.7251 ori_loss: 10.7251 l1_error: 0.000000
ms_loss: 10.7820 ori_loss: 1

ms_loss: 10.7820 ori_loss: 10.7820 l1_error: 0.000000
ms_loss: 10.6864 ori_loss: 10.6864 l1_error: 0.000000
ms_loss: 10.7614 ori_loss: 10.7614 l1_error: 0.000000
ms_loss: 10.7166 ori_loss: 10.7166 l1_error: 0.000000
ms_loss: 10.7781 ori_loss: 10.7781 l1_error: 0.000000
ms_loss: 10.7970 ori_loss: 10.7970 l1_error: 0.000000
ms_loss: 10.7694 ori_loss: 10.7694 l1_error: 0.000000
ms_loss: 10.7701 ori_loss: 10.7701 l1_error: 0.000000
ms_loss: 10.7303 ori_loss: 10.7303 l1_error: 0.000000
ms_loss: 10.7937 ori_loss: 10.7937 l1_error: 0.000000
ms_loss: 10.8167 ori_loss: 10.8167 l1_error: 0.000001
ms_loss: 10.7595 ori_loss: 10.7595 l1_error: 0.000000
ms_loss: 10.7964 ori_loss: 10.7964 l1_error: 0.000001
ms_loss: 10.7279 ori_loss: 10.7279 l1_error: 0.000000
ms_loss: 10.6673 ori_loss: 10.6673 l1_error: 0.000000
ms_loss: 10.7662 ori_loss: 10.7662 l1_error: 0.000000
ms_loss: 10.7186 ori_loss: 10.7186 l1_error: 0.000000
ms_loss: 10.7410 ori_loss: 10.7410 l1_error: 0.000002
ms_loss: 10.8144 ori_loss: 1

ms_loss: 10.6749 ori_loss: 10.6749 l1_error: 0.000001
ms_loss: 10.7678 ori_loss: 10.7678 l1_error: 0.000000
ms_loss: 10.7231 ori_loss: 10.7231 l1_error: 0.000000
ms_loss: 10.7549 ori_loss: 10.7549 l1_error: 0.000000
ms_loss: 10.6931 ori_loss: 10.6931 l1_error: 0.000001
ms_loss: 10.7437 ori_loss: 10.7437 l1_error: 0.000000
ms_loss: 10.7163 ori_loss: 10.7163 l1_error: 0.000001
ms_loss: 10.7545 ori_loss: 10.7545 l1_error: 0.000000
ms_loss: 10.7260 ori_loss: 10.7260 l1_error: 0.000000
ms_loss: 10.7669 ori_loss: 10.7669 l1_error: 0.000000
ms_loss: 10.6364 ori_loss: 10.6364 l1_error: 0.000001
ms_loss: 10.6867 ori_loss: 10.6867 l1_error: 0.000000
ms_loss: 10.7168 ori_loss: 10.7168 l1_error: 0.000000
ms_loss: 10.8642 ori_loss: 10.8642 l1_error: 0.000001
ms_loss: 10.7370 ori_loss: 10.7370 l1_error: 0.000000
ms_loss: 10.6926 ori_loss: 10.6926 l1_error: 0.000000
ms_loss: 10.7416 ori_loss: 10.7416 l1_error: 0.000000
ms_loss: 10.7700 ori_loss: 10.7700 l1_error: 0.000001
ms_loss: 10.7570 ori_loss: 1

ms_loss: 10.7756 ori_loss: 10.7756 l1_error: 0.000000
ms_loss: 10.7293 ori_loss: 10.7293 l1_error: 0.000000
ms_loss: 10.7405 ori_loss: 10.7405 l1_error: 0.000000
ms_loss: 10.6938 ori_loss: 10.6938 l1_error: 0.000000
ms_loss: 10.7008 ori_loss: 10.7008 l1_error: 0.000001
ms_loss: 10.7833 ori_loss: 10.7833 l1_error: 0.000000
ms_loss: 10.7137 ori_loss: 10.7137 l1_error: 0.000001
ms_loss: 10.8068 ori_loss: 10.8068 l1_error: 0.000000
ms_loss: 10.6620 ori_loss: 10.6620 l1_error: 0.000000
ms_loss: 10.8509 ori_loss: 10.8509 l1_error: 0.000000
ms_loss: 10.7950 ori_loss: 10.7950 l1_error: 0.000000
ms_loss: 10.7608 ori_loss: 10.7608 l1_error: 0.000001
ms_loss: 10.7485 ori_loss: 10.7485 l1_error: 0.000000
ms_loss: 10.6435 ori_loss: 10.6435 l1_error: 0.000001
ms_loss: 10.6802 ori_loss: 10.6802 l1_error: 0.000000
ms_loss: 10.7693 ori_loss: 10.7693 l1_error: 0.000000
ms_loss: 10.6925 ori_loss: 10.6925 l1_error: 0.000000
ms_loss: 10.7824 ori_loss: 10.7824 l1_error: 0.000001
ms_loss: 10.6805 ori_loss: 1

In [33]:
axes = ops.stack((y_x, y), axis=0).T

In [40]:
from mindspore import Tensor
inds = Tensor([])

In [17]:
inds2 = Tensor([1, 2])

In [18]:
a = Tensor([[1, 3, 5], [2, 4, 6], [3, 6, 8]])

In [23]:
xx = Tensor(((0,1), (1,2)))

In [32]:
a[[[0,1], [1,2]]]

Tensor(shape=[2], dtype=Int64, value= [3, 6])

In [41]:
a[inds]

Tensor(shape=[1, 2, 2, 3], dtype=Int64, value=
[[[[1, 3, 5],
   [2, 4, 6]],
  [[2, 4, 6],
   [3, 6, 8]]]])

In [62]:
_gather = P.Gather()

In [65]:
_gather(a, inds, -1)

Tensor(shape=[3, 2, 2], dtype=Int64, value=
[[[1, 3],
  [3, 5]],
 [[2, 4],
  [4, 6]],
 [[3, 6],
  [6, 8]]])

In [60]:
for ind in inds:
    print(ind)
    print(a[ind])

[0 1]
[[1 3 5]
 [2 4 6]]
[1 2]
[[2 4 6]
 [3 6 8]]


In [55]:
a_i = P.Concat(axis=0)([a[ind] for ind in inds])

In [56]:
a_i

Tensor(shape=[4, 2], dtype=Int64, value=
[[1, 3],
 [2, 4],
 [2, 4],
 [3, 6]])

In [49]:
a[inds2]

Tensor(shape=[2, 2], dtype=Int64, value=
[[2, 4],
 [3, 6]])

In [66]:
y1.shape

(992, 1)

In [67]:
y1

Tensor(shape=[992, 1], dtype=Int32, value=
[[1037],
 [4743],
 [7719],
 ...
 [-100],
 [-100],
 [-100]])

In [31]:
y1[0]

Tensor(shape=[1], dtype=Int32, value= [1037])

In [51]:
loss = _gather_d(x, target_dim, y1)

In [52]:
loss

Tensor(shape=[992, 1], dtype=Float32, value=
[[ 9.77265418e-01],
 [ 6.47866130e-01],
 [ 7.58817554e-01],
 ...
 [ 1.80141628e-02],
 [-6.06357396e-01],
 [ 5.73751628e-02]])

In [71]:
loss2 = _gather_d2(x, target_dim, y1)

In [72]:
loss2

Tensor(shape=[992, 1], dtype=Float32, value=
[[ 9.77265418e-01],
 [ 6.47866130e-01],
 [ 7.58817554e-01],
 ...
 [ 1.80141628e-02],
 [-6.06357396e-01],
 [ 5.73751628e-02]])

In [50]:
_gather = P.Gather()

In [30]:
x

Tensor(shape=[992, 30523], dtype=Float32, value=
[[ 4.32402939e-02,  1.72222644e-01, -2.72406727e-01 ...  4.83820736e-01, -1.63055420e-01,  4.67659026e-01],
 [ 3.64985555e-01, -7.27577627e-01,  1.46366447e-01 ...  3.22835207e-01, -2.29929969e-01, -4.27669406e-01],
 [-5.47032356e-01,  1.43672317e-01, -2.06260383e-02 ...  2.38050371e-01, -2.35057652e-01,  2.05625534e-01],
 ...
 [ 6.30636215e-01,  2.92517602e-01, -3.93146753e-01 ... -4.71520424e-02,  4.35345054e-01, -7.17348099e-01],
 [ 5.66837251e-01, -2.24138916e-01, -8.57901335e-01 ... -4.89547193e-01, -4.35822397e-01, -1.05204093e+00],
 [ 6.53074265e-01, -2.94384956e-01, -6.31686032e-01 ...  2.86771655e-01, -6.35821670e-02, -2.80281723e-01]])

In [32]:
loss = _gather(x, y1, 0)

[ERROR] DEVICE(3434083,fffe37f64290,python):2023-10-10-11:02:47.117.680 [mindspore/ccsrc/plugin/device/ascend/hal/device/ascend_kernel_runtime.cc:675] TaskFailCallback] Execute TaskFailCallback failed. task_fail_info or current_graph_ is nullptr
[ERROR] DEVICE(3434083,fffe37f64290,python):2023-10-10-11:02:47.117.782 [mindspore/ccsrc/plugin/device/ascend/hal/device/ascend_stream_manager.cc:186] SyncStream] Call runtime rtStreamSynchronize error.
[ERROR] DEVICE(3434083,fffe37f64290,python):2023-10-10-11:02:47.117.796 [mindspore/ccsrc/plugin/device/ascend/hal/device/ascend_kernel_runtime.cc:1127] SyncStream] Sync default stream failed.


RuntimeError: Sync stream error!

----------------------------------------------------
- C++ Call Stack: (For framework developers)
----------------------------------------------------
mindspore/ccsrc/plugin/device/ascend/hal/device/ascend_device_address.cc:203 SyncStream


In [23]:
loss

Tensor(shape=[992, 992, 1], dtype=Float32, value=
[[[ 9.77265418e-01],
  [ 1.79878339e-01],
  [ 5.28340578e-01],
  ...
  [ 0.00000000e+00],
  [ 0.00000000e+00],
  [ 0.00000000e+00]],
 [[ 1.43269628e-01],
  [ 6.47866130e-01],
  [ 1.35163844e+00],
  ...
  [-3.70816290e-01],
  [-3.70816290e-01],
  [-3.70816290e-01]],
 [[ 1.92740276e-01],
  [ 9.82333869e-02],
  [ 7.58817554e-01],
  ...
  [-7.43796885e-01],
  [-7.43796885e-01],
  [-7.43796885e-01]],
 ...
 [[ 8.45621347e-01],
  [ 1.16952825e+00],
  [ 1.81982666e-02],
  ...
  [-3.02402735e-01],
  [-3.02402735e-01],
  [-3.02402735e-01]],
 [[ 1.55460566e-01],
  [ 4.58987981e-01],
  [ 4.10465956e-01],
  ...
  [ 1.80141628e-02],
  [ 1.80141628e-02],
  [ 1.80141628e-02]],
 [[ 8.60769510e-01],
  [ 8.50641847e-01],
  [ 8.54853928e-01],
  ...
  [-6.06357396e-01],
  [-6.06357396e-01],
  [-6.06357396e-01]]])

In [25]:
mask = ops.ones(992, ms.float32)

In [26]:
loss1(x, y, mask)

Tensor(shape=[1], dtype=Float32, value= [ 8.83468628e+00])

In [16]:
import torch
from torch.nn import CrossEntropyLoss as torch_CrossEntropyLoss

In [22]:
x_torch = torch.from_numpy(x.asnumpy())
y_torch = torch.from_numpy(y.asnumpy()).to(torch.long)

In [23]:
loss_torch = torch_CrossEntropyLoss(label_smoothing=0.1)

In [24]:
loss_torch(x_torch, y_torch)

tensor(10.2473)

In [6]:
from mindformers.models.blip2.qformer import CrossEntropyLoss as CrossEntropyLoss2

In [7]:
loss2 = CrossEntropyLoss2(label_smoothing=0.1)

In [8]:
loss2(x, y)

Tensor(shape=[], dtype=Float32, value= 10.2473)

In [9]:
from mindspore.nn import CrossEntropyLoss as CrossEntropyLoss3

In [10]:
loss3 = CrossEntropyLoss3(label_smoothing=0.1)

In [11]:
loss3(x, y)

Tensor(shape=[], dtype=Float32, value= 9.52042)

In [41]:
coid = 10
round_n = 1000

In [56]:
coid = 0
round_n = 1000
for i in range(round_n):
    x = msnp.randn(992, 30523)
    y = msnp.randn(992).astype(ms.int32)
    l2 = loss2(x, y)
    l3 = loss3(x, y)
    if l2 == l3:
        print(True)
        coid += 1
    else:
        print(False)
        print(l2, l3)

print(f"coincide rate: {(coid/round_n):.3f}")

True
True
True
True
True
True
True
True
True
False
10.799058 10.799057
True
True
True
True
True
True
False
10.821322 10.821323
True
True
True
True
True
True
True
True
True
True
True
False
10.845504 10.845503
True
True
True
False
10.769189 10.76919
False
10.850328 10.8503275
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
10.828471 10.82847
True
True
False
10.862382 10.862383
False
10.874336 10.874337
True
True
False
10.80709 10.807089
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
10.800744 10.800745
True
True
True
False
10.829005 10.829004
False
10.806739 10.806741
True
True
True
True
False
10.802286 10.802287
True
True
True
True
False
10.843796 10.843795
True
True
True
False
10.77926 10.779259
True
False
10.809778 10.809777
False
10.807304 10.807303
True
True
True
True
True
True
True
False
10.878033 10.878031
True
True
True
True
True
True
False
10.826295 10.826293
True
False
10.833531 10.83353

False
10.826852 10.826851
False
10.825918 10.825919
False
10.816288 10.816287
False
10.882231 10.882232
True
True
True
False
10.843797 10.843796
True
True
True
True
True
True
True
True
True
True
True
True
True
False
10.821589 10.82159
True
True
False
10.799782 10.799781
True
coincide rate: 0.834
